In [1]:
#import library

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from collections import defaultdict
import math
import os

In this research we use the ionic conductivity dataset taken from https://github.com/FALL-ML/materials-discovery. The formula names in the dataset have irregular formats, so we have changed them all to the same format. We have combined stoichiometry on the same elements and have changed so that the stoichiometric ratio are to the right of the element. For example 0.7Li2S-0.3P2S5 is changed to Li1.4S2.2P0.6.	

In [2]:
#read dataset
dataset = pd.read_csv('D:/RESEARCH/230313 Machine Learning for SSE/FINAL_CODE/dataset/original/revised_13xx_v3.3.csv', encoding= 'unicode_escape')


dataset_org = dataset_13xx.copy()

In [3]:
dataset.columns

Index(['id', 'formula', 'target_', 'Ea (eV)', 'space group', 'ICSD Entry',
       'helper_toedit', 'helper_todelete', 'questionable',
       'formula_stoi_converted', 'single_phase_target'],
      dtype='object')

In [4]:
dataset.columns = ['id', 'formula_', 'target_', 'Ea (eV)', 'space group', 'ICSD Entry',
       'helper_toedit', 'helper_todelete', 'questionable',
       'formula_stoi_converted', 'single_phase_target']

Discard data that has uncertain ionic conductivity. For example '<-10'. This data has a string type, different from most data which has a float type

In [5]:
#drop target with string type

dataset = dataset[pd.to_numeric(dataset['target_'], errors='coerce').notnull()]
dataset['target_'] = dataset['target_'].astype(float)

In [6]:
dataset

,id,formula_,target_,Ea (eV),space group,ICSD Entry,helper_toedit,helper_todelete,questionable,formula_stoi_converted,single_phase_target
0,1,LiAlSi3O8,1.300000e-10,NaN,2.0,81980.0,0,0,0,LiAlSi3O8,1
1,2,LiSn2(PO4)3,2.040000e-09,NaN,2.0,83832.0,0,0,0,LiSn2(PO4)3,1
2,3,Li7BiO6,8.800000e-07,0.580,2.0,155950.0,0,0,0,Li7BiO6,1
3,4,Li7SbO6,6.700000e-08,0.700,2.0,413370.0,0,0,0,Li7SbO6,1
4,5,Li7P3S11,1.700000e-02,0.170,2.0,157654.0,0,0,0,Li7P3S11,1
...,...,...,...,...,...,...,...,...,...,...,...
1341,1342,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,2.260000e-04,0.390,NaN,NaN,0,0,0,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,1
1342,1343,Li2ZrS3,7.300000e-06,NaN,NaN,NaN,0,0,0,Li2ZrS3,1
1343,1344,Li2.2Zn0.1Zr0.9S3,1.200000e-04,NaN,NaN,NaN,0,0,0,Li2.2Zn0.1Zr0.9S3,1
1344,1345,0.7Li2S-0.3P2S5,8.100000e-05,0.425,NaN,NaN,1,0,0,Li1.4S2.2P0.6,0


In [16]:
#convert target to log scale

def calc_log(list_val):
    final_values = []
    for v in list_val:
        final_values.append(math.log10(v))
    return final_values

list_log = calc_log(dataset_13xx['target_'])

dataset_13xx['target']=list_log
dataset_13xx

,id,formula_,target_,Ea (eV),space group,ICSD Entry,helper_toedit,helper_todelete,questionable,formula_stoi_converted,single_phase_target,target
0,1,LiAlSi3O8,1.300000e-10,NaN,2.0,81980.0,0,0,0,LiAlSi3O8,1,-9.886057
1,2,LiSn2(PO4)3,2.040000e-09,NaN,2.0,83832.0,0,0,0,LiSn2(PO4)3,1,-8.690370
2,3,Li7BiO6,8.800000e-07,0.580,2.0,155950.0,0,0,0,Li7BiO6,1,-6.055517
3,4,Li7SbO6,6.700000e-08,0.700,2.0,413370.0,0,0,0,Li7SbO6,1,-7.173925
4,5,Li7P3S11,1.700000e-02,0.170,2.0,157654.0,0,0,0,Li7P3S11,1,-1.769551
...,...,...,...,...,...,...,...,...,...,...,...,...
1341,1342,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,2.260000e-04,0.390,NaN,NaN,0,0,0,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,1,-3.645892
1342,1343,Li2ZrS3,7.300000e-06,NaN,NaN,NaN,0,0,0,Li2ZrS3,1,-5.136677
1343,1344,Li2.2Zn0.1Zr0.9S3,1.200000e-04,NaN,NaN,NaN,0,0,0,Li2.2Zn0.1Zr0.9S3,1,-3.920819
1344,1345,0.7Li2S-0.3P2S5,8.100000e-05,0.425,NaN,NaN,1,0,0,Li1.4S2.2P0.6,0,-4.091515


#### Open grouped elements. 
For example LiSn2(PO4)3 to LiSn2P3O12


In [17]:
import importlib
from open_bracket import *
# importlib.reload(open_bracket)

dataset["formula"] = dataset["formula_stoi_converted"].apply(lambda x:open_bracket_multi(x))
dataset
# open_bracket_multi('As200(PO4)0.5((PO2)0.2Be0.5(CO)0.2)2')

,id,formula_,target_,Ea (eV),space group,ICSD Entry,helper_toedit,helper_todelete,questionable,formula_stoi_converted,single_phase_target,target,formula
0,1,LiAlSi3O8,1.300000e-10,NaN,2.0,81980.0,0,0,0,LiAlSi3O8,1,-9.886057,LiAlSi3O8
1,2,LiSn2(PO4)3,2.040000e-09,NaN,2.0,83832.0,0,0,0,LiSn2(PO4)3,1,-8.690370,LiSn2P3O12
2,3,Li7BiO6,8.800000e-07,0.580,2.0,155950.0,0,0,0,Li7BiO6,1,-6.055517,Li7BiO6
3,4,Li7SbO6,6.700000e-08,0.700,2.0,413370.0,0,0,0,Li7SbO6,1,-7.173925,Li7SbO6
4,5,Li7P3S11,1.700000e-02,0.170,2.0,157654.0,0,0,0,Li7P3S11,1,-1.769551,Li7P3S11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,1342,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,2.260000e-04,0.390,NaN,NaN,0,0,0,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,1,-3.645892,Li6.6La2.5Y0.5Zr1.6Ta0.4O12
1342,1343,Li2ZrS3,7.300000e-06,NaN,NaN,NaN,0,0,0,Li2ZrS3,1,-5.136677,Li2ZrS3
1343,1344,Li2.2Zn0.1Zr0.9S3,1.200000e-04,NaN,NaN,NaN,0,0,0,Li2.2Zn0.1Zr0.9S3,1,-3.920819,Li2.2Zn0.1Zr0.9S3
1344,1345,0.7Li2S-0.3P2S5,8.100000e-05,0.425,NaN,NaN,1,0,0,Li1.4S2.2P0.6,0,-4.091515,Li1.4S2.2P0.6


#### There are hundreds of duplicate element. The duplicate element are combined with average.



In [59]:
duplicate_data = dataset_13xx[dataset_13xx.duplicated(['formula'], keep=False)]

keys = list(duplicate_data.columns.values)
i1 = dataset_13xx.set_index(keys).index
i2 = duplicate_data.set_index(keys).index

non_duplicate_data = dataset_13xx[~i1.isin(i2)]

In [60]:
duplicate_data_ = duplicate_data.loc[:,['formula', 'target']]
duplicate_data_

,formula,target
4,Li7P3S11,-1.769551
5,Li7P3S11,-2.494850
14,Li7P3S11,-2.065502
16,Li3.8Ge0.8P0.2S4,-5.749580
17,Li3.6Ge0.6P0.4S4,-3.756962
...,...,...
1330,Li1.5Sc0.5Ti1.5P3O12,-6.266001
1336,LiTi2P3O12,-5.143876
1338,Li6.6La3Zr1.6Ta0.4O12,-3.504456
1344,Li1.4S2.2P0.6,-4.091515


In [61]:
duplicate_data_grouped = duplicate_data_.groupby(['formula']).mean()
duplicate_data_grouped['formula'] = duplicate_data_grouped.index
duplicate_data_grouped = duplicate_data_grouped.reset_index(drop=True)
duplicate_data_grouped

,target,formula
0,-2.778603,I0.3Li1.24P0.18S1.2Si0.14
1,-3.577451,La0.51Li0.34TiO2.94
2,-3.202646,La0.52Li0.45TiO3
3,-2.529914,La0.54Li0.36TiO3
4,-3.044754,La0.54Li0.39TiO3
...,...,...
129,-4.453637,LiS1.5Ge0.5
130,-5.060051,LiS1.5Si0.5
131,-4.972964,LiTi2P3O12
132,-7.230193,LiZr2P3O12


In [63]:
duplicate_data = duplicate_data[~duplicate_data.duplicated(['formula'], keep='first')]
duplicate_data

,id,formula_,target_,Ea (eV),space group,ICSD Entry,helper_toedit,helper_todelete,questionable,formula_stoi_converted,single_phase_target,target,formula
4,5,Li7P3S11,0.017000,0.170,2.0,157654.0,0,0,0,Li7P3S11,1,-1.769551,Li7P3S11
16,17,Li3.8Ge0.8P0.2S4,0.000002,0.470,11.0,NaN,0,0,0,Li3.8Ge0.8P0.2S4,1,-5.749580,Li3.8Ge0.8P0.2S4
17,18,Li3.6Ge0.6P0.4S4,0.000175,0.340,11.0,NaN,0,0,0,Li3.6Ge0.6P0.4S4,1,-3.756962,Li3.6Ge0.6P0.4S4
18,19,Li3.4Ge0.4P0.6S4,0.000654,0.280,11.0,NaN,0,0,0,Li3.4Ge0.4P0.6S4,1,-3.184422,Li3.4Ge0.4P0.6S4
19,20,Li3.35Ge0.35P0.65S4,0.001540,0.230,11.0,NaN,0,0,0,Li3.35Ge0.35P0.65S4,1,-2.812479,Li3.35Ge0.35P0.65S4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,982,0.75Li2O-0.25P2S5,0.000018,0.480,NaN,NaN,1,0,0,Li1.5O0.75P0.5S1.25,0,-4.744727,Li1.5O0.75P0.5S1.25
1013,1014,0.2LiI-0.8(0.6Li2S-0.4SiS2),0.000352,NaN,NaN,NaN,1,0,0,Li1.160I0.200S1.280Si0.400,0,-3.453457,Li1.16I0.2S1.28Si0.4
1014,1015,0.3LiI-0.7(0.6Li2S-0.4SiS2),0.000742,NaN,NaN,NaN,1,0,0,Li1.140I0.300S1.220Si0.400,0,-3.129596,Li1.14I0.3S1.22Si0.4
1015,1016,0.5Li2S-0.5SiS2,0.000002,NaN,NaN,NaN,1,0,0,Li1.0S1.5Si0.5,0,-5.801343,LiS1.5Si0.5


In [64]:
processed_duplicate_data = pd.merge(duplicate_data_grouped, duplicate_data, left_on='formula', right_on='formula', how='left')
processed_duplicate_data

,target_x,formula,id,formula_,target_,Ea (eV),space group,ICSD Entry,helper_toedit,helper_todelete,questionable,formula_stoi_converted,single_phase_target,target_y
0,-2.778603,I0.3Li1.24P0.18S1.2Si0.14,867,0.14 SiS2-0.09 P2S5-0.47 Li2S-0.30 LiI,1.320000e-03,0.34,NaN,NaN,1,0,0,I0.3Li1.24P0.18S1.2Si0.14,0,-2.879426
1,-3.577451,La0.51Li0.34TiO2.94,564,La0.51Li0.34TiO2.94,7.000000e-05,0.36,221.0,NaN,0,0,0,La0.51Li0.34TiO2.94,1,-4.154902
2,-3.202646,La0.52Li0.45TiO3,200,La0.52Li0.45TiO3,5.010000e-04,NaN,123.0,50434.0,0,0,0,La0.52Li0.45TiO3,1,-3.300162
3,-2.529914,La0.54Li0.36TiO3,550,La0.54Li0.36TiO3,9.790000e-03,NaN,221.0,NaN,0,0,0,La0.54Li0.36TiO3,1,-2.009217
4,-3.044754,La0.54Li0.39TiO3,199,La0.54Li0.39TiO3,6.510000e-04,NaN,123.0,NaN,0,0,0,La0.54Li0.39TiO3,1,-3.186419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,-4.453637,LiS1.5Ge0.5,791,0.5(Li2S) 0.5(GeS2),3.300000e-05,0.35,NaN,NaN,1,0,0,Li1.0S1.5Ge0.5,0,-4.481486
130,-5.060051,LiS1.5Si0.5,1016,0.5Li2S-0.5SiS2,1.580000e-06,NaN,NaN,NaN,1,0,0,Li1.0S1.5Si0.5,0,-5.801343
131,-4.972964,LiTi2P3O12,297,LiTi2(PO4)3,1.610000e-04,0.21,148.0,NaN,0,0,0,LiTi2(PO4)3,1,-3.793174
132,-7.230193,LiZr2P3O12,310,LiZr2(PO4)3,2.960000e-10,NaN,167.0,201935.0,0,0,0,LiZr2(PO4)3,1,-9.528708


In [66]:
final_duplicate_data = processed_duplicate_data.loc[:, ['formula_', 'formula', 'space group', 'target_x']]
final_duplicate_data.columns = ['formula_', 'formula', 'space group', 'target']
final_nonduplicate_data = non_duplicate_data.loc[:, ['formula_', 'formula', 'space group', 'target']]

In [67]:
final_data = pd.concat([final_duplicate_data, final_nonduplicate_data])
final_data

,formula_,formula,space group,target
0,0.14 SiS2-0.09 P2S5-0.47 Li2S-0.30 LiI,I0.3Li1.24P0.18S1.2Si0.14,NaN,-2.778603
1,La0.51Li0.34TiO2.94,La0.51Li0.34TiO2.94,221.0,-3.577451
2,La0.52Li0.45TiO3,La0.52Li0.45TiO3,123.0,-3.202646
3,La0.54Li0.36TiO3,La0.54Li0.36TiO3,221.0,-2.529914
4,La0.54Li0.39TiO3,La0.54Li0.39TiO3,123.0,-3.044754
...,...,...,...,...
1339,Li6.6La2.875Y0.125Zr1.6Ta0.4O12,Li6.6La2.875Y0.125Zr1.6Ta0.4O12,NaN,-3.498941
1340,Li6.6La2.75Y0.25Zr1.6Ta0.4O12,Li6.6La2.75Y0.25Zr1.6Ta0.4O12,NaN,-3.360514
1341,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,Li6.6La2.5Y0.5Zr1.6Ta0.4O12,NaN,-3.645892
1342,Li2ZrS3,Li2ZrS3,NaN,-5.136677


### Split Dataset 5 Fold CV

In [69]:
def create_k_fold(data, url):
    kf = KFold(n_splits=5, shuffle=True, random_state=123)

    train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=123)
  # test_num = round(len(data)*0.2)
  # test=data[len(data)-test_num:] #20% for test data
  # train=data[:len(data)-test_num] #remaining data -> kfold

    kf.get_n_splits(train)
    for id, (train_index, val_index) in enumerate(kf.split(train)):
        i=id+1
        if not os.path.exists(url+ str(i)):
            os.makedirs(url+ str(i))
        train.iloc[train_index].to_csv(url+ str(i)+ '/train.csv')
        train.iloc[val_index].to_csv(url+ str(i)+ '/val.csv')
        test.to_csv(url+ str(i)+ '/test.csv')

In [70]:
# create_k_fold(dataset_13xx,'D:/RESEARCH/230313 Machine Learning for SSE/FINAL_CODE/dataset/k_fold/')
# create_k_fold(final_data,'D:/RESEARCH/230313 Machine Learning for SSE/FINAL_CODE/dataset/k_fold_230110/')